In [1]:
import pandas as pd

In [2]:
import boto3
session = boto3.Session(profile_name='iismturk', region_name='us-east-1')
mtc = session.client('mturk')
def get_all_hits():
    return [hit for page in mtc.get_paginator('list_hits').paginate() for hit in page['HITs']]
all_hits = get_all_hits()

In [3]:
transactions = pd.read_csv('/Users/kcarnold/Downloads/Transactions_2018-03-01_to_2018-05-01.csv')

In [4]:
hit_data = pd.DataFrame(all_hits).drop(['Question'], axis=1)

In [5]:
transactions.columns

Index(['Transaction ID', 'Date Initiated', 'Date Posted', 'Transaction Type',
       'Payment Method', 'Recipient ID', 'Amount', 'HIT ID', 'Assignment ID'],
      dtype='object')

In [6]:
hit_data.columns

Index(['AssignmentDurationInSeconds', 'AutoApprovalDelayInSeconds',
       'CreationTime', 'Description', 'Expiration', 'HITGroupId', 'HITId',
       'HITLayoutId', 'HITReviewStatus', 'HITStatus', 'HITTypeId', 'Keywords',
       'MaxAssignments', 'NumberOfAssignmentsAvailable',
       'NumberOfAssignmentsCompleted', 'NumberOfAssignmentsPending',
       'QualificationRequirements', 'RequesterAnnotation', 'Reward', 'Title'],
      dtype='object')

In [28]:
def download_gsheet(sheet_url):
    return pd.read_csv(sheet_url.replace('/edit#gid=', '/export?format=csv&gid='))
charge_to = download_gsheet('https://docs.google.com/spreadsheets/d/1ynVK0oxVN98Ej3EdqlmYgdE-tkoEUKwgWeBO4hTorM0/edit#gid=382794845')

In [29]:
hit_data['Title'] = hit_data['Title'].str.strip()
charge_to['Title'] = charge_to['Title'].str.strip()

In [30]:
hits_with_chargeto = pd.merge(
    hit_data,
    charge_to,
    on='Title',
    how='left', validate='m:1', indicator=True)
hits_with_chargeto['_merge'].value_counts()

both          240
right_only      0
left_only       0
Name: _merge, dtype: int64

In [31]:
hits_with_chargeto[hits_with_chargeto['_merge'] == 'left_only'].Title

Series([], Name: Title, dtype: object)

In [36]:
hit_ids = hits_with_chargeto[hits_with_chargeto['ChargeTo'] == 'Ken-Writing'].HITId

In [43]:
import tqdm

In [100]:
def get_assignments(hit_id):
    return [
        assignment
        for page in mtc.get_paginator('list_assignments_for_hit').paginate(HITId=hit_id)
        for assignment in page['Assignments']]

assignments = [
    assignment
    for hit_id in tqdm.tqdm_notebook(hits_with_chargeto[hits_with_chargeto['ChargeTo'] == 'Ken-Anno'].HITId)
    for assignment in get_assignments(hit_id)
]

In [101]:
df_assignments = pd.DataFrame(assignments)

In [102]:
import xmltodict

In [107]:
def force_list(x):
    if not isinstance(x, list): return [x]
    
df_assignments = pd.concat(
    [
        df_assignments,
        pd.DataFrame([
            dict([(q['QuestionIdentifier'], q['FreeText']) for q in force_list(xmltodict.parse(answer)['QuestionFormAnswers']['Answer'])])
            for answer in df_assignments['Answer']])
    ], axis=1)

TypeError: 'NoneType' object is not iterable

In [104]:
df_assignments[df_assignments.duplicated(subset=['WorkerId'], keep=False)]

,AcceptTime,Answer,ApprovalTime,AssignmentId,AssignmentStatus,AutoApprovalTime,HITId,SubmitTime,WorkerId
0,2018-05-14 14:36:04-04:00,"<?xml version=""1.0"" encoding=""ASCII""?><Questio...",NaT,3EF8EXOTT1V6RV28Z2LGX6HZ83JJ1E,Submitted,2018-05-17 14:36:47-04:00,3ZFRE2BDQ9E2VRUCYF47RPD23D3XZK,2018-05-14 14:36:47-04:00,A177EXELDLWTWV
1,2018-05-14 14:37:00-04:00,"<?xml version=""1.0"" encoding=""ASCII""?><Questio...",NaT,37XITHEISW97CMVJN4J9GTMHMH4CRP,Submitted,2018-05-17 14:39:22-04:00,3ZFRE2BDQ9E2VRUCYF47RPD23D3XZK,2018-05-14 14:39:22-04:00,A0017268R9SKD8U2Y3F
2,2018-05-14 14:37:08-04:00,"<?xml version=""1.0"" encoding=""ASCII""?><Questio...",NaT,3HPZF4IVNMTG9N751ZB3OPFEP4JCYR,Submitted,2018-05-17 14:43:40-04:00,3ZFRE2BDQ9E2VRUCYF47RPD23D3XZK,2018-05-14 14:43:40-04:00,A37AJI03M37NPJ
3,2018-05-14 14:36:59-04:00,"<?xml version=""1.0"" encoding=""ASCII""?><Questio...",NaT,3TPZPLC3M0CYN9JLHJ4JIVI8FM3P3C,Submitted,2018-05-17 14:38:36-04:00,3MGHRFQY2LPUTBEWZHDH44HPNCEY0Z,2018-05-14 14:38:36-04:00,A0017268R9SKD8U2Y3F
4,2018-05-14 14:37:04-04:00,"<?xml version=""1.0"" encoding=""ASCII""?><Questio...",NaT,3TVSS0C0E10T6ZEG8KO5S2U22IJWTV,Submitted,2018-05-17 14:40:36-04:00,3MGHRFQY2LPUTBEWZHDH44HPNCEY0Z,2018-05-14 14:40:36-04:00,A110KENBXU7SUJ
5,2018-05-14 14:37:07-04:00,"<?xml version=""1.0"" encoding=""ASCII""?><Questio...",NaT,32SCWG5HIH4XKS630L2HPYUSPD5P6I,Submitted,2018-05-17 14:41:21-04:00,3MGHRFQY2LPUTBEWZHDH44HPNCEY0Z,2018-05-14 14:41:21-04:00,A3O81LHBBI8NPK
6,2018-05-14 14:37:18-04:00,"<?xml version=""1.0"" encoding=""ASCII""?><Questio...",NaT,3RWE2M8QWHA23W8BIH98C8F6CVIN0J,Submitted,2018-05-17 14:42:46-04:00,36AZSFEYZ40E94M1SMNMHM9EWZ9VB9,2018-05-14 14:42:46-04:00,A110KENBXU7SUJ
7,2018-05-14 14:38:17-04:00,"<?xml version=""1.0"" encoding=""ASCII""?><Questio...",NaT,3JNQLM5FT4MJB684LRX7I44W4Y32LH,Submitted,2018-05-17 14:43:21-04:00,36AZSFEYZ40E94M1SMNMHM9EWZ9VB9,2018-05-14 14:43:21-04:00,A1DXFMUVK5015A
8,2018-05-14 14:36:47-04:00,"<?xml version=""1.0"" encoding=""ASCII""?><Questio...",NaT,351SEKWQS0HQKYO5ISI3E0AP1UHMDB,Submitted,2018-05-17 14:55:30-04:00,36AZSFEYZ40E94M1SMNMHM9EWZ9VB9,2018-05-14 14:55:30-04:00,A1D3QRG52OISSR
9,2018-05-14 14:36:23-04:00,"<?xml version=""1.0"" encoding=""ASCII""?><Questio...",NaT,34S6N1K2ZVJNQWBM4C0E58AYTHKHLD,Submitted,2018-05-17 14:36:32-04:00,3TLFH2L6Y9O58ZIG4YUWFBIP58H2TZ,2018-05-14 14:36:32-04:00,A36SM7QM8OK3H6


In [81]:
list(set(df_assignments.WorkerId))

['A13HZKB2NGZP6B',
 'A1IXH2QLUU5BKZ',
 'AKLV0WIZZ356X',
 'A3LAVDGDP66YF4',
 'A3MFK5UEUG95M3',
 'A35F9DZWLLHFY4',
 'AADRVG2QVZH75',
 'AK7ZL6VR82SEB',
 'A2O7H7VXLFN6BP',
 'A1XTRYSSOR598O',
 'A2JP9IKRHNLRPI',
 'A1CH3TODZNQCES',
 'A3QEVFM3UD67BA',
 'A34FOMRRI5KYCY',
 'A1ZT30BGR3266K',
 'ACN0C9HPYDTED',
 'A2GB0RI43KU4Z',
 'A1F9KLZGHE9DTA',
 'AOKTAHY4F4J2B',
 'A2VRDE2FHCBMF8',
 'AU7A3QNJF3O00',
 'A1GXFMAC759VRM',
 'A1LTJHPUTL7WM9',
 'A5NHP0N1XC09K',
 'A25CFVQND4A0EV',
 'A3GPTSDDHJ2DBV',
 'A10BH9PYCYUKDJ',
 'A377LTGWJKY2IW']

In [86]:
qualification = mtc.list_qualification_types(MustBeRequestable=False, Query='did-captioning')['QualificationTypes']

In [94]:
qualification[0]

{'AutoGranted': False,
 'CreationTime': datetime.datetime(2018, 5, 16, 13, 34, 48, tzinfo=tzlocal()),
 'Description': 'Has completed the image captioning study.',
 'IsRequestable': True,
 'Name': 'did-captioning',
 'QualificationTypeId': '39S8MMYHZL2XNIVBIHDYC6EOK2DX1K',
 'QualificationTypeStatus': 'Active'}

In [90]:
qualification_id = qualification[0]['QualificationTypeId']

In [97]:
# pd.DataFrame([{'Worker ID': worker_id, 'UPDATE-did-captioning': 1} for worker_id in set(df_assignments.WorkerId)]).to_csv('update_qualifications.csv')

In [108]:
for worker_id in tqdm.tqdm_notebook(set(df_assignments.WorkerId)):
    mtc.associate_qualification_with_worker(QualificationTypeId=qualification_id, WorkerId=worker_id, IntegerValue=1, SendNotification=False)

In [33]:
annotated_HITs = pd.merge(
    transactions[transactions['Transaction Type'] != 'Prepayment'],
    hits_with_chargeto,
    left_on='HIT ID',
    right_on='HITId',
    how='left')

In [9]:
annotated_HITs[annotated_HITs['Title'].isna()]

,Transaction ID,Date Initiated,Date Posted,Transaction Type,Payment Method,Recipient ID,Amount,HIT ID,Assignment ID,AssignmentDurationInSeconds,...,HITTypeId,Keywords,MaxAssignments,NumberOfAssignmentsAvailable,NumberOfAssignmentsCompleted,NumberOfAssignmentsPending,QualificationRequirements,RequesterAnnotation,Reward,Title


In [10]:
annotated_HITs[annotated_HITs['Transaction ID'].isna()]

,Transaction ID,Date Initiated,Date Posted,Transaction Type,Payment Method,Recipient ID,Amount,HIT ID,Assignment ID,AssignmentDurationInSeconds,...,HITTypeId,Keywords,MaxAssignments,NumberOfAssignmentsAvailable,NumberOfAssignmentsCompleted,NumberOfAssignmentsPending,QualificationRequirements,RequesterAnnotation,Reward,Title


In [11]:
annotated_HITs.groupby('Title').CreationTime.aggregate(['min', 'max'])

,min,max
Title,,
Create a face using shapes [duration: < 3 minutes],2018-03-09 11:32:31-05:00,2018-04-09 05:05:54-04:00
"Create a face using shapes [duration: < 6 minutes, if you have done this HIT before, please refrain from doing it again]",2018-04-16 18:09:49-04:00,2018-04-16 18:09:49-04:00
Create a face using shapes [duration: < 6 minutes],2018-04-10 13:20:46-04:00,2018-04-10 13:20:46-04:00
Take our VR experiment,2018-03-01 19:42:30-05:00,2018-03-08 18:09:46-05:00
Take our VR experiment (SMARTPHONE REQUIRED),2018-03-09 14:36:38-05:00,2018-03-09 14:36:38-05:00
Take our VR experiment (TABLET REQUIRED),2018-03-09 14:39:11-05:00,2018-03-09 14:39:11-05:00
Try writing with different mobile-phone keyboards,2018-04-09 18:33:30-04:00,2018-04-27 09:22:10-04:00
Which image is more creative? [<1 minutes],2018-04-10 18:39:59-04:00,2018-04-17 09:41:08-04:00
Which image is more creative? [<5 minutes],2018-03-24 21:50:17-04:00,2018-03-27 18:09:27-04:00


In [12]:
total_amounts = (-annotated_HITs.groupby('Title')['Amount'].sum().sort_values().to_frame('total_amount'))
total_amounts

,total_amount
Title,
Create a face using shapes [duration: < 3 minutes],136.8
Try writing with different mobile-phone keyboards,75.6
Which image is more creative? [<5 minutes],38.0
"Create a face using shapes [duration: < 6 minutes, if you have done this HIT before, please refrain from doing it again]",37.8
Which image is more creative? [<1 minutes],32.2
Take our VR experiment,29.4
Create a face using shapes [duration: < 6 minutes],25.2
Take our VR experiment (SMARTPHONE REQUIRED),13.5
Take our VR experiment (TABLET REQUIRED),13.5


In [21]:
charge_to

,Title,ChargeTo
0,Create a face using shapes [duration: < 3 minu...,Yuna
1,Try writing with different mobile-phone keyboards,Ken-Writing
2,Which image is more creative? [<5 minutes],Yuna
3,Create a face using shapes [duration: < 6 minu...,Yuna
4,Which image is more creative? [<1 minutes],Yuna
5,Take our VR experiment,Bernd
6,Create a face using shapes [duration: < 6 minu...,Yuna
7,Take our VR experiment (SMARTPHONE REQUIRED),Bernd
8,Take our VR experiment (TABLET REQUIRED),Bernd
9,Pick the image that this sentence is more like...,Ken-Anno


In [71]:
hits_with_chargeto = pd.merge(
    annotated_HITs,
    charge_to,
    on='Title',
    how='left', validate='m:1')#, indicator=True)
    

In [73]:
# hits_with_chargeto._merge.value_counts()

In [74]:
assert hits_with_chargeto.ChargeTo.isna().sum() == 0

In [77]:
transactions[transactions['HIT ID'].isin(hits_with_chargeto[hits_with_chargeto['ChargeTo'] == 'Ken']['HIT ID'])]['Amount'].sum()

-75.6